<a href="https://colab.research.google.com/github/armandordorica/APS1070_A1/blob/master/TTT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

import numpy as np
import copy 
import random

winning_sequences = [[(0,0), (0,1), (0,2)], [(1,0), (1,1), (1,2)], [(2,0), (2,1), (2,2)], \
                     [(0,0), (1,0), (2,0)], [(0,1), (1,1), (2,1)], [(0,2), (1,2), (2,2)], \
                     [(0,0), (1,1), (2,2)],[(2,0), (1,1), (0,2)]]


class Game: 
  def __init__(self, grid, player1, player2):
    self.grid = grid
    self.player1 = player1
    self.player2 = player2


  def play_computer(self): 
    if len(self.get_viable_winning_sequences_for_computer())>0: 
      viable_winning_choices = list(set(sum(human_comp_game.get_viable_winning_sequences_for_computer(), [])) - set(human_comp_game.grid.os_positions))

      print(f"viable_winning_choices for computer:{viable_winning_choices}") 
      random_comp_choice = random.choice(viable_winning_choices)
      (i,j) = random_comp_choice
      print(f"computer attempt to play at: {(i,j)}")
      self.grid.update_value(i,j,self.player2.sign)
    else: 
      random_comp_choice = random.choice(self.grid.available_positions)
      (i,j) = random_comp_choice
      print(f"computer attempt to play at: {(i,j)}")

      self.grid.update_value(i,j,self.player2.sign)

  def get_viable_winning_sequences_for_computer(self): 
    possible_winning_sequences = []

    for i in range(0,len(self.grid.os_positions)): 
      for j in range(0,len(winning_sequences)): 
        if self.grid.os_positions[i] in winning_sequences[j]: 
          possible_winning_sequences.append(winning_sequences[j])

    viable_winning_sequences = []
    for i in range(0,len(possible_winning_sequences)): 
      if len(set(possible_winning_sequences[i]).intersection(set(human_comp_game.grid.xs_positions))) ==0: 
        viable_winning_sequences.append(possible_winning_sequences[i])
    
    return viable_winning_sequences

  

class Player: 
  def __init__(self, sign, is_human=True):
    self.is_human = is_human
    self.sign = sign
    
  


class Grid: 
  def __init__(self):
    self.grid = [['', '', ''], ['', '', ''], ['', '', '']]
    self.queue = []
    self.available_positions = self.get_available_positions()
    self.os_positions = []
    self.xs_positions = []

  def print_grid(self): 
    for x in self.grid: 
      print(x)

  def reset_board(self): 
    self.grid = [['', '', ''], ['', '', ''], ['', '', '']]
    self.temp_grid = [['', '', ''], ['', '', ''], ['', '', '']]
    self.queue = []

  def update_value(self, i, j, value): 
    if self.check_move_validity(i,j,value)==True: 
        if self.grid[i][j] == '':
          ### Either no one has gone or no repeated player
          if (len(self.queue)>0 and self.queue[-1]!=value) or len(self.queue)==0:
            print("update_value - move is valid")
            self.grid[i][j] = value
            self.available_positions = self.get_available_positions()
            self.queue.append(value)
            self.os_positions = self.get_sign_positions('O')
            self.xs_positions = self.get_sign_positions('X')

        else: 
          print(f"The grid at position [i][j] is not empty or wrong turn")
    self.print_grid()

    self.get_next_player_allowed()

    is_there_a_win = self.check_win()

    if is_there_a_win[0] == True: 
      print(f"{is_there_a_win[2]} has won")
      print("GAME OVER")
    
    else: 
      print("No winners yet")
    
    
    if self.check_full_board() ==True  and is_there_a_win[0] ==False: 
      print("This is a Draw. Game Over. ")
    

  def get_sign_positions(self,sign): 
    sign_positions = []

    for i in range(3): 
      for j in range(3): 
        if self.grid[i][j] ==sign: 
          sign_positions.append((i,j))

    return sign_positions



  def check_win(self): 
    win = 0
    winning_sequence = None
    winning_string = None
    winning_player = None
    for i in range(0,len(winning_sequences)): 
      # print(f"sequence:{winning_sequences[i]}")
      s = ''
      for j in range(0,3): 
        position = winning_sequences[i][j]
        # print(f"position:{position}")
        x = position[0]
        y = position[1]

        char_at_pos = self.grid[x][y]
        s+=char_at_pos

      # print(s)

      if len(s) == 3: 
        # print(f"WIN with sequence[{winning_sequences[i]}]")

        if s.count('X') == 3: 
          win = 1
          winning_sequence = winning_sequences[i]
          winning_string = s
          winning_player = 'X'
        
        elif s.count('O') == 3: 
          win = 1
          winning_sequence = winning_sequences[i]
          winning_string = s
          winning_player = 'O'

    if win==1: 
      print(f"Winning_sequence: {winning_sequence}")
      print(f"Winning Player: {winning_player}")
      print(f"Winning String: {winning_string}")
      return True, winning_sequence, winning_player 
    else: 
      print(f"No winning sequence")
      return False, winning_sequence, winning_string


  def check_move_validity(self, i,j, value): 
    self.temp_grid = copy.deepcopy(self.grid)
    # print("before update")
    # print(f"self.grid:{self.grid}")
    # print(f"self.temp_grid:{self.temp_grid}")
    self.temp_grid[i][j] = value

    # print("after update")
    # print(f"self.grid:{self.grid}")
    # print(f"self.temp_grid:{self.temp_grid}")

    xs = ''
    os = ''
    for i in range(0,len(self.temp_grid)):#rows
      for j in range(0,len(self.temp_grid[0])): #columns
        # print(i,j)
        char = self.temp_grid[i][j]

        if char =='X': 
          xs+='X'
        
        elif char =='O': 
          os+='O'
        
        elif char !='O' and char!= 'X' and len(char)>0: 
          print(f"Character {char} is invalid") 

    num_xs = len(xs)
    num_os = len(os)
    # print(f"num_xs: {num_xs}")
    # print(f"num_os: {num_os}")

    if np.abs(num_xs - num_os) > 1: 
      print("Invalid board. Someone skipped turns")
      return False
    else: 
      print("Board is valid.")
      return True

  def check_full_board(self): 
    xs = ''
    os = ''
    for i in range(0,len(self.temp_grid)):#rows
      for j in range(0,len(self.temp_grid[0])): #columns
        # print(i,j)
        char = self.temp_grid[i][j]

        if char =='X': 
          xs+='X'
        
        elif char =='O': 
          os+='O'
        
        elif char !='O' and char!= 'X' and len(char)>0: 
          print(f"Character {char} is invalid") 

    num_xs = len(xs)
    num_os = len(os)

    # print(num_xs), print(num_os)

    if num_xs + num_os == 9: 
      print("Board is full")
      return True
    else: 
      return False 

  def get_next_player_allowed(self): 
    
    if len(self.queue) == 0: 
      next_player_allowed = 'X'

    elif len(self.queue)>0 and self.queue[-1] == 'X': 
      next_player_allowed = 'O'
      print(f"Last player was: {self.queue[-1]}")

    elif len(self.queue)>0 and self.queue[-1] == 'O': 
      next_player_allowed = 'X'
      print(f"Last player was: {self.queue[-1]}")

    else: 
      print("Error in input ")
      next_player_allowed = False 

    print(f"Next player allowed is: {next_player_allowed} ")
    return next_player_allowed
  
  def get_available_positions(self): 
    possible_positions = [(i,j) for i in range(3) for j in range(3)]

    filled_positions = []
    for i in range(0,len(self.grid)): 
      for j in range(0,len(self.grid)): 
        if len(self.grid[i][j])>0: 
          filled_positions.append((i,j))


    available_positions = list(set(possible_positions) -set(filled_positions))
    available_positions.sort()
    
    return available_positions


      

class Cell: 
  def __init__(self, i,j, value, grid):
    self.i = i
    self.j = j
    self.value = value
    self.grid = grid  



### Initialization of the Grid

In [2]:
ttt = Grid()

In [3]:
ttt.print_grid()

['', '', '']
['', '', '']
['', '', '']


In [4]:
ttt.available_positions

[(0, 0), (0, 1), (0, 2), (1, 0), (1, 1), (1, 2), (2, 0), (2, 1), (2, 2)]

In [5]:
ttt.update_value(0,0,'X')

Board is valid.
update_value - move is valid
['X', '', '']
['', '', '']
['', '', '']
Last player was: X
Next player allowed is: O 
No winning sequence
No winners yet


In [6]:
ttt.available_positions

[(0, 1), (0, 2), (1, 0), (1, 1), (1, 2), (2, 0), (2, 1), (2, 2)]

In [7]:
ttt.queue

['X']

### Testing for invalid moves

In [8]:
ttt.update_value(1,1,'X')

Invalid board. Someone skipped turns
['X', '', '']
['', '', '']
['', '', '']
Last player was: X
Next player allowed is: O 
No winning sequence
No winners yet


In [9]:
ttt.available_positions

[(0, 1), (0, 2), (1, 0), (1, 1), (1, 2), (2, 0), (2, 1), (2, 2)]

In [10]:
ttt.queue

['X']

### Verifying that state of board was not altered after invalid move

In [11]:
ttt.print_grid()

['X', '', '']
['', '', '']
['', '', '']


In [12]:
ttt.update_value(2,2,'X')

Invalid board. Someone skipped turns
['X', '', '']
['', '', '']
['', '', '']
Last player was: X
Next player allowed is: O 
No winning sequence
No winners yet


### Verifying that board is updated with valid move following invalid move 

In [13]:
ttt.update_value(2,2,'O')

Board is valid.
update_value - move is valid
['X', '', '']
['', '', '']
['', '', 'O']
Last player was: O
Next player allowed is: X 
No winning sequence
No winners yet


In [14]:
ttt.print_grid()

['X', '', '']
['', '', '']
['', '', 'O']


In [15]:
ttt.check_win()

No winning sequence


(False, None, None)

In [16]:
ttt.update_value(0,1,'X')

Board is valid.
update_value - move is valid
['X', 'X', '']
['', '', '']
['', '', 'O']
Last player was: X
Next player allowed is: O 
No winning sequence
No winners yet


In [17]:
ttt.check_win()

No winning sequence


(False, None, None)

In [18]:
ttt.update_value(2,1,'O')

Board is valid.
update_value - move is valid
['X', 'X', '']
['', '', '']
['', 'O', 'O']
Last player was: O
Next player allowed is: X 
No winning sequence
No winners yet


### Verifying that game alerts when the game is over and someone has won

In [19]:
ttt.update_value(0,2,'X')

Board is valid.
update_value - move is valid
['X', 'X', 'X']
['', '', '']
['', 'O', 'O']
Last player was: X
Next player allowed is: O 
Winning_sequence: [(0, 0), (0, 1), (0, 2)]
Winning Player: X
Winning String: XXX
X has won
GAME OVER


In [20]:
ttt.check_win()

Winning_sequence: [(0, 0), (0, 1), (0, 2)]
Winning Player: X
Winning String: XXX


(True, [(0, 0), (0, 1), (0, 2)], 'X')

### Reset Functionality 

In [21]:
ttt.reset_board()
ttt.print_grid()

['', '', '']
['', '', '']
['', '', '']


In [22]:
ttt.grid


[['', '', ''], ['', '', ''], ['', '', '']]

### Testing for the case of a draw

In [23]:
ttt.reset_board()
ttt.print_grid()


['', '', '']
['', '', '']
['', '', '']


In [24]:
ttt.update_value(2,1,'O')

Board is valid.
update_value - move is valid
['', '', '']
['', '', '']
['', 'O', '']
Last player was: O
Next player allowed is: X 
No winning sequence
No winners yet


In [25]:
ttt.update_value(0,0,'X')

Board is valid.
update_value - move is valid
['X', '', '']
['', '', '']
['', 'O', '']
Last player was: X
Next player allowed is: O 
No winning sequence
No winners yet


In [26]:
ttt.update_value(0,1,'O')

Board is valid.
update_value - move is valid
['X', 'O', '']
['', '', '']
['', 'O', '']
Last player was: O
Next player allowed is: X 
No winning sequence
No winners yet


In [27]:
ttt.update_value(1,1,'X')

Board is valid.
update_value - move is valid
['X', 'O', '']
['', 'X', '']
['', 'O', '']
Last player was: X
Next player allowed is: O 
No winning sequence
No winners yet


In [28]:
ttt.update_value(2,2,'O')

Board is valid.
update_value - move is valid
['X', 'O', '']
['', 'X', '']
['', 'O', 'O']
Last player was: O
Next player allowed is: X 
No winning sequence
No winners yet


In [29]:
ttt.update_value(2,0,'X')

Board is valid.
update_value - move is valid
['X', 'O', '']
['', 'X', '']
['X', 'O', 'O']
Last player was: X
Next player allowed is: O 
No winning sequence
No winners yet


In [30]:
ttt.update_value(1,0,'O')

Board is valid.
update_value - move is valid
['X', 'O', '']
['O', 'X', '']
['X', 'O', 'O']
Last player was: O
Next player allowed is: X 
No winning sequence
No winners yet


In [31]:
ttt.update_value(1,2,'X')

Board is valid.
update_value - move is valid
['X', 'O', '']
['O', 'X', 'X']
['X', 'O', 'O']
Last player was: X
Next player allowed is: O 
No winning sequence
No winners yet


In [32]:
ttt.update_value(0,2,'O')

Board is valid.
update_value - move is valid
['X', 'O', 'O']
['O', 'X', 'X']
['X', 'O', 'O']
Last player was: O
Next player allowed is: X 
No winning sequence
No winners yet
Board is full
This is a Draw. Game Over. 


In [33]:
ttt.grid

[['X', 'O', 'O'], ['O', 'X', 'X'], ['X', 'O', 'O']]

In [34]:
ttt.check_win()[0]

No winning sequence


False

In [35]:
ttt.check_full_board()

Board is full


True

### Human Player vs Computer using random positioning only 

In [36]:
human_player = Player('X', True)
computer_player = Player('O', False)

In [37]:
ttt.reset_board()
ttt.print_grid()


['', '', '']
['', '', '']
['', '', '']


In [38]:
human_player.is_human

True

In [39]:
computer_player.is_human

False

In [40]:
human_comp_game = Game(ttt, human_player, computer_player)

In [41]:
human_comp_game.grid.print_grid()

['', '', '']
['', '', '']
['', '', '']


### Human goes first

In [42]:
human_comp_game.grid.print_grid()

['', '', '']
['', '', '']
['', '', '']


In [43]:
human_comp_game.grid.update_value(0,1,human_comp_game.player1.sign)

Board is valid.
update_value - move is valid
['', 'X', '']
['', '', '']
['', '', '']
Last player was: X
Next player allowed is: O 
No winning sequence
No winners yet


In [44]:
human_comp_game.grid

In [45]:
human_comp_game.grid

### Checking that one player cannot overwrite the board if there is something in it already

In [46]:
human_comp_game.grid.update_value(0,1,human_comp_game.player2.sign)

Board is valid.
The grid at position [i][j] is not empty or wrong turn
['', 'X', '']
['', '', '']
['', '', '']
Last player was: X
Next player allowed is: O 
No winning sequence
No winners yet


In [47]:
human_comp_game.grid.update_value(0,1,human_comp_game.player2.sign)

Board is valid.
The grid at position [i][j] is not empty or wrong turn
['', 'X', '']
['', '', '']
['', '', '']
Last player was: X
Next player allowed is: O 
No winning sequence
No winners yet


In [48]:
human_comp_game.player2.sign

'O'

In [49]:
human_comp_game.grid.update_value(0,1,human_comp_game.player2.sign)

Board is valid.
The grid at position [i][j] is not empty or wrong turn
['', 'X', '']
['', '', '']
['', '', '']
Last player was: X
Next player allowed is: O 
No winning sequence
No winners yet


In [50]:
human_comp_game.grid.available_positions

[(0, 0), (0, 2), (1, 0), (1, 1), (1, 2), (2, 0), (2, 1), (2, 2)]

In [51]:
human_comp_game.grid.get_next_player_allowed()

Last player was: X
Next player allowed is: O 


'O'

### Computer's Turn

In [52]:
human_comp_game.play_computer()

computer attempt to play at: (1, 2)
Board is valid.
update_value - move is valid
['', 'X', '']
['', '', 'O']
['', '', '']
Last player was: O
Next player allowed is: X 
No winning sequence
No winners yet


In [53]:
human_comp_game.grid.queue

['X', 'O']

### Human's Turn

In [54]:
human_comp_game.grid.update_value(2,1,human_comp_game.player1.sign)

Board is valid.
update_value - move is valid
['', 'X', '']
['', '', 'O']
['', 'X', '']
Last player was: X
Next player allowed is: O 
No winning sequence
No winners yet


### Computer's turn

In [55]:
human_comp_game.play_computer()

viable_winning_choices for computer:[(1, 0), (1, 1), (2, 2), (0, 2)]
computer attempt to play at: (2, 2)
Board is valid.
update_value - move is valid
['', 'X', '']
['', '', 'O']
['', 'X', 'O']
Last player was: O
Next player allowed is: X 
No winning sequence
No winners yet


### Human's Turn

In [56]:
human_comp_game.grid.update_value(2,2,human_comp_game.player1.sign)

Invalid board. Someone skipped turns
['', 'X', '']
['', '', 'O']
['', 'X', 'O']
Last player was: O
Next player allowed is: X 
No winning sequence
No winners yet


### Computer's Turn

In [57]:
human_comp_game.play_computer()

viable_winning_choices for computer:[(1, 0), (0, 2), (1, 1), (0, 0)]
computer attempt to play at: (1, 1)
Board is valid.
['', 'X', '']
['', '', 'O']
['', 'X', 'O']
Last player was: O
Next player allowed is: X 
No winning sequence
No winners yet


### Human's Turn

In [58]:
human_comp_game.grid.update_value(0,0,human_comp_game.player1.sign)

Board is valid.
update_value - move is valid
['X', 'X', '']
['', '', 'O']
['', 'X', 'O']
Last player was: X
Next player allowed is: O 
No winning sequence
No winners yet


### Computer's Turn

In [59]:
human_comp_game.play_computer()

viable_winning_choices for computer:[(1, 0), (1, 1), (0, 2)]
computer attempt to play at: (1, 1)
Board is valid.
update_value - move is valid
['X', 'X', '']
['', 'O', 'O']
['', 'X', 'O']
Last player was: O
Next player allowed is: X 
No winning sequence
No winners yet


In [60]:
human_comp_game.grid.update_value(0,2,human_comp_game.player1.sign)

Board is valid.
update_value - move is valid
['X', 'X', 'X']
['', 'O', 'O']
['', 'X', 'O']
Last player was: X
Next player allowed is: O 
Winning_sequence: [(0, 0), (0, 1), (0, 2)]
Winning Player: X
Winning String: XXX
X has won
GAME OVER


In [61]:
is_game_over = human_comp_game.grid.check_win()[0]
is_game_over

Winning_sequence: [(0, 0), (0, 1), (0, 2)]
Winning Player: X
Winning String: XXX


True

### Running the Game

In [69]:
ttt = Grid()
ttt.print_grid()

human_player = Player('X', True)
computer_player = Player('O', False)

human_comp_game = Game(ttt, human_player, computer_player)

is_game_over = False

counter = 0
while not human_comp_game.grid.check_win()[0] and counter<100: 
  print(f"Round {counter+1}...")
  # print("This is what the current board looks like: ")
  # human_comp_game.grid.print_grid()

  print(f"Hello human player.")
  print(f"Enter i coordinate where you want to place your X")
  i= input()
  print(f"Enter j coordinate where you want to place your X")
  j= input()

  print("Human Playing...")
  human_comp_game.grid.update_value(int(i),int(j),human_comp_game.player1.sign)

  is_game_over = human_comp_game.grid.check_win()[0]
  print(f"Status of the game is {is_game_over}")

  if is_game_over: 
    break 

  print("\n")
  print("Computer Playing...")
  human_comp_game.play_computer()

  if is_game_over: 
    break 

  print("\n")
  counter+=1


['', '', '']
['', '', '']
['', '', '']
No winning sequence
Round 1...
Hello human player.
Enter i coordinate where you want to place your X
0
Enter j coordinate where you want to place your X
0
Human Playing...
Board is valid.
update_value - move is valid
['X', '', '']
['', '', '']
['', '', '']
Last player was: X
Next player allowed is: O 
No winning sequence
No winners yet
No winning sequence
Status of the game is False


Computer Playing...
computer attempt to play at: (1, 2)
Board is valid.
update_value - move is valid
['X', '', '']
['', '', 'O']
['', '', '']
Last player was: O
Next player allowed is: X 
No winning sequence
No winners yet


No winning sequence
Round 2...
Hello human player.
Enter i coordinate where you want to place your X
2
Enter j coordinate where you want to place your X
2
Human Playing...
Board is valid.
update_value - move is valid
['X', '', '']
['', '', 'O']
['', '', 'X']
Last player was: X
Next player allowed is: O 
No winning sequence
No winners yet
No winni